In [1]:
ours2orig = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 254, 41, 42, 43, 44, 45,
46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 85,
86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103,
104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 136, 138, 132,
133, 134, 135, 220, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 185, 186,
187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
217, 218, 219, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232,
233, 137, 234, 235, 236, 237, 238, 239, 240, 241, 250, 251, 255, 256, 257,
258, 259, 260, 261, 262, 263, 242, 243, 244, 245, 0, 1, 2, 3, 4, 5, 6, 7, 8,
9, 10, 11, 83, 84, 131, 139, 140, 141, 181, 182, 183, 184, 246, 247, 248,
249, 252, 253]

def vec2mat(v):
    a,b = np.triu_indices(264,1)
    m = np.zeros((264,264))
    m[a,b] = v
    return m+m.T

def remap(fc, roimap=ours2orig):
    fc = fc[roimap,:]
    fc = fc[:,roimap]
    return fc

print('Complete')

Complete


In [2]:
# Load FC

import pickle
import numpy as np

pncdir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/PNC/'
demo = pickle.load(open(f'{pncdir}/demographics.pkl', 'rb'))

rest = []
nback = []
emoid = []
race = []
sex = []
age = []
wrat = []
subids = []

a,b = np.triu_indices(264,1)

for sub in demo['age_at_cnb']:
    try:
        ra = demo['Race'][sub]
        ag = demo['age_at_cnb'][sub]
        se = demo['Sex'][sub]
        if ra not in ['AA', 'EA']:
            continue
        ra = ra == 'AA'
        se = se == 'M'
        r = np.load(f'{pncdir}/fc/{sub}_task-rest_fc.npy')
        n = np.load(f'{pncdir}/fc/{sub}_task-nback_fc.npy')
        e = np.load(f'{pncdir}/fc/{sub}_task-emoid_fc.npy')
        race.append(ra)
        sex.append(se)
        age.append(ag)
        rest.append(remap(vec2mat(r))[a,b])
        nback.append(remap(vec2mat(n))[a,b])
        emoid.append(remap(vec2mat(e))[a,b])
        subids.append(sub)
    except:
        pass

rest = np.stack(rest)
nback = np.stack(nback)
emoid = np.stack(emoid)
race = np.array(race).astype('int')
sex = np.array(sex).astype('int')
age = np.array(age)

age = (age - np.mean(age)) / np.std(age)

print([a.shape for a in [rest, nback, emoid, race, sex, age]])

[(1193, 34716), (1193, 34716), (1193, 34716), (1193,), (1193,), (1193,)]


In [14]:
restfilt = pickle.load(open('/home/anton/Documents/Tulane/Research/PNC_Good/PNC_SPM_rest_good_filt.pkl', 'rb'))
nbackfilt = pickle.load(open('/home/anton/Documents/Tulane/Research/PNC_Good/PNC_SPM_nback_good_filt.pkl', 'rb'))
emoidfilt = pickle.load(open('/home/anton/Documents/Tulane/Research/PNC_Good/PNC_SPM_emoid_good_filt.pkl', 'rb'))

restc = []
nbackc = []
emoidc = []

c,d = np.triu_indices(264)

def cov_vec2mat(cov):
    c,d = np.triu_indices(264)
    if len(cov.shape) == 1:
        I = np.zeros((264,264))
        I[c,d] = cov
        I[d,c] = cov
    if len(cov.shape) == 2:
        I = np.zeros((cov.shape[0],264,264))
        I[:,c,d] = cov
        I[:,d,c] = cov
    return I

for sub in subids:
    r = restfilt[sub][ours2orig,:]
    r = r@r.T
    n = nbackfilt[sub][ours2orig,:]
    n = n@n.T
    e = emoidfilt[sub][ours2orig,:]
    e = e@e.T
    restc.append(r[c,d])
    nbackc.append(n[c,d])
    emoidc.append(e[c,d])

restc = np.stack(restc)
nbackc = np.stack(nbackc)
emoidc = np.stack(emoidc)

print([a.shape for a in [restc, nbackc, emoidc]])

[(1193, 34980), (1193, 34980), (1193, 34980)]


In [15]:
m = cov_vec2mat(restc[0])
diag = 1/np.sqrt(np.diag(m))
p = m*np.expand_dims(diag, 0)*np.expand_dims(diag, 1)
print(p)
print(vec2mat(rest[0]))

[[1.         0.31330028 0.17657666 ... 0.21176661 0.33170009 0.1886001 ]
 [0.31330028 1.         0.06662559 ... 0.10477732 0.14424741 0.09979316]
 [0.17657666 0.06662559 1.         ... 0.28855774 0.39245903 0.30750799]
 ...
 [0.21176661 0.10477732 0.28855774 ... 1.         0.23334039 0.03120363]
 [0.33170009 0.14424741 0.39245903 ... 0.23334039 1.         0.649513  ]
 [0.1886001  0.09979316 0.30750799 ... 0.03120363 0.649513   1.        ]]
[[0.         0.31651752 0.17764036 ... 0.21048194 0.33388131 0.19089991]
 [0.31651752 0.         0.06645934 ... 0.10557357 0.14390005 0.09924158]
 [0.17764036 0.06645934 0.         ... 0.2888381  0.39240873 0.30743745]
 ...
 [0.21048194 0.10557357 0.2888381  ... 0.         0.23382944 0.03178571]
 [0.33388131 0.14390005 0.39240873 ... 0.23382944 0.         0.64941332]
 [0.19089991 0.09924158 0.30743745 ... 0.03178571 0.64941332 0.        ]]


In [20]:
import pickle

mygen3 = pickle.load(open('/home/anton/Documents/Tulane/Research/ImageNomer/data/PNC/mygen4.pkl', 'rb'))

print('Done')

Done


In [21]:
import torch

age_w = torch.from_numpy(mygen3['age_w']).float().cuda()
age_i = mygen3['age_i']
sex_w = torch.from_numpy(mygen3['sex_w']).float().cuda()
sex_i = mygen3['sex_i']
race_w = torch.from_numpy(mygen3['race_w']).float().cuda()
race_i = mygen3['race_i']
rest_w = torch.from_numpy(mygen3['rest_w']).float().cuda()
rest_i = mygen3['rest_i']
nback_w = torch.from_numpy(mygen3['nback_w']).float().cuda()
nback_i = mygen3['nback_i']
emoid_w = torch.from_numpy(mygen3['emoid_w']).float().cuda()
emoid_i = mygen3['emoid_i']

print(age_w.shape, age_i, sex_w.shape, sex_i, race_w.shape, race_i)

torch.Size([34716]) -0.11856799409128756 torch.Size([34716]) -5.628305026005052 torch.Size([34716]) 13.770769923836573


In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import random

class VAE(nn.Module):
    def __init__(self, fcd, ld):
        super(VAE, self).__init__()
        self.fcd = fcd
        self.ld = ld
        self.enc1 = nn.Linear(fcd, 1000).float().cuda()
        self.enc2 = nn.Linear(1000, ld).float().cuda()
        self.dec1 = nn.Linear(ld+6, 1000).float().cuda()
        self.dec2 = nn.Linear(1000, fcd).float().cuda()

    def enc(self, x):
        x = F.relu(self.enc1(x))
        z = self.enc2(x)
        return z

    def gen(self, n):
        return torch.randn(n, self.ld).float().cuda()/(10**0.5)
    
    def dec(self, z, age, sex, race, rest, nback, emoid):
        z = torch.cat([z, age.unsqueeze(1), sex.unsqueeze(1), race.unsqueeze(1), 
                       rest.unsqueeze(1), nback.unsqueeze(1), emoid.unsqueeze(1)], dim=1)
        x = F.relu(self.dec1(z))
        x = self.dec2(x)
        return x

def latent_loss(z):
    C = z.T@z
    mu = torch.mean(z, dim=0)
    tgt1 = torch.eye(z.shape[-1]).float().cuda()*len(z)/10
    tgt2 = torch.zeros(z.shape[-1]).float().cuda()
    lossC = rmse(C, tgt1)
    lossmu = rmse(mu, tgt2)
    return lossC, lossmu, C, mu

def decor_loss(z, age, sex, race):
    age = age - torch.mean(age)
    sex = sex - torch.mean(sex)
    race = race - torch.mean(race)
    agep = torch.einsum('n,nz->z', age, z)
    sexp = torch.einsum('n,nz->z', sex, z)
    racep = torch.einsum('n,nz->z', race, z)
    tgt = torch.zeros(z.shape[-1]).float().cuda()
    loss_age = rmse(agep, tgt)
    loss_sex = rmse(sexp, tgt)
    loss_race = rmse(racep, tgt)
    return loss_age + loss_sex + loss_race

def rmse(a, b, mean=torch.mean):
    return mean((a-b)**2)**0.5

def pretty(x):
    return f'{round(float(x), 4)}'

vae = VAE(restc.shape[-1], 30)
optim = torch.optim.Adam(vae.parameters(), lr=5e-5, weight_decay=0)

rest_t = torch.from_numpy(restc).float().cuda()
nback_t = torch.from_numpy(nbackc).float().cuda()
emoid_t = torch.from_numpy(emoidc).float().cuda()
age_t = torch.from_numpy(age).float().cuda()
sex_t = torch.from_numpy(sex).float().cuda()
race_t = torch.from_numpy(race).float().cuda()

nepochs = 3000
pperiod = 50

ce = nn.CrossEntropyLoss()

for e in range(nepochs):
    if e % 3 == 0:
        tidx = 0
        x = rest_t/10000
    elif e % 3 == 1:
        tidx = 1
        x = nback_t/10000
    else:
        tidx = 2
        x = emoid_t/10000
    task = torch.ones(len(x)).long().cuda()
    # Reconstruct
    optim.zero_grad()
    z = vae.enc(x)
    y = vae.dec(z, age_t, sex_t, race_t, task*(tidx == 0), task*(tidx == 1), task*(tidx == 2))
    lossC, lossmu, _, _ = latent_loss(z)
    lossDecor = decor_loss(z, age_t, sex_t, race_t)
    lossrec = rmse(x, y)
    # Generate
    tidx = random.randint(0,2)
    ag = torch.ones(100).float().cuda()*torch.randn(1)[0]
    sx = torch.ones(100).long().cuda()*random.randint(0,1)
    rc = torch.ones(100).long().cuda()*random.randint(0,1)
    task = torch.ones(100).long().cuda()
    z = vae.gen(100)
    y = vae.dec(z, ag, sx, rc, task*(tidx == 0), task*(tidx == 1), task*(tidx == 2))
    y = y*10000
    I = torch.zeros(y.shape[0], 264, 264).float().cuda()
    c,d = np.triu_indices(264)
    I[:,c,d] = y
    I[:,d,c] = y
    ee = np.arange(264)
    diag = 1/I[:,ee,ee]
    I = torch.einsum('abc,ab->abc',I,diag)
    I = torch.einsum('abc,ac->abc',I,diag)
    a,b = np.triu_indices(264,1)
    y = I[:,a,b]
    ag_hat = y@age_w + age_i
    sx_hat = y@sex_w + sex_i
    rc_hat = y@race_w + race_i
    rs_hat = y@rest_w + rest_i
    nb_hat = y@nback_w + nback_i
    em_hat = y@emoid_w + emoid_i
    age_loss = rmse(ag_hat, ag)
    sex_loss = ce(torch.stack([-sx_hat, sx_hat], dim=1), sx)
    race_loss = ce(torch.stack([-rc_hat, rc_hat], dim=1), rc)
    rest_loss = ce(torch.stack([-rs_hat, rs_hat], dim=1), task*(tidx == 0))
    nback_loss = ce(torch.stack([-nb_hat, nb_hat], dim=1), task*(tidx == 1))
    emoid_loss = ce(torch.stack([-em_hat, em_hat], dim=1), task*(tidx == 2))
    (lossC+lossmu+lossrec+lossDecor+0.1*age_loss+0.1*sex_loss+0.1*race_loss+0.1*rest_loss+0.1*nback_loss+0.1*emoid_loss).backward()
    optim.step()
    if e%pperiod == 0 or e == nepochs-1:
        print(f'{e} {pretty(lossC)} {pretty(lossmu)} {pretty(lossDecor)} *{pretty(lossrec)}* {pretty(age_loss)} {pretty(sex_loss)} {pretty(race_loss)}' + 
             f' {pretty(rest_loss)} {pretty(nback_loss)} {pretty(emoid_loss)}')

print('Done')

tensor([-1.3842e+01,  1.3117e+01, -1.1930e+01, -1.0170e+01, -1.2121e+01,
        -6.7168e+00, -1.2169e+01,  7.0029e+01, -7.4235e+00,  2.5851e+01,
        -9.6002e+00, -1.1817e+01, -2.6159e+00, -9.7742e+00, -8.9676e+00,
        -1.2674e+01, -1.1186e+01, -1.1939e+01, -1.5223e+01, -1.7521e+01,
        -1.3519e+01, -8.2830e+01, -3.8276e+00,  7.7002e+00,  1.2723e+03,
        -9.6353e+00, -8.2471e+00, -1.0166e+01,  5.7714e+01, -7.5887e+00,
        -1.5432e+01, -9.0085e+00, -7.3033e+00, -9.7495e+00, -9.7163e+00,
        -1.2509e+01, -1.0649e+01, -2.5427e+01, -9.4898e+02, -2.9785e+00,
        -7.4965e+00, -1.4711e+01, -5.5360e+00, -4.3184e+00, -3.9385e+00,
        -7.0310e+00, -3.9540e+01, -9.8099e+00, -3.2395e+03, -7.7459e+00,
        -7.6456e+01, -9.5130e+00, -8.7974e+00, -3.4345e+01, -1.1757e+01,
        -2.8587e+01, -1.0983e+01, -1.2871e+01, -1.0500e+01, -7.9633e+00,
        -1.1005e+01, -1.1457e+01, -3.6852e+01, -8.1535e+00, -9.5230e+00,
        -2.4131e+01, -1.0196e+01, -9.7873e+00, -1.9

KeyboardInterrupt: 